In [8]:
import json
from collections import Counter
from pathlib import Path
import pandas as pd  # 요약표 출력을 위해 pandas 사용

# 1. 특정 폴더 경로 설정
folder_path = Path('/Users/seunghyunmoon/Code/Toy/vote_player_server/data/candidates')  # <-- 여기를 본인 폴더로 설정하세요

# 결과 기록용 리스트
summary_records = []
empty_bills_files = []  # bills 없는 파일 목록

# 2. 폴더 내 모든 JSON 파일 순회
json_files = list(folder_path.glob('*.json'))

print(f"총 {len(json_files)}개의 JSON 파일을 처리합니다.\n")

for file_path in json_files:
    print(f"처리 중: {file_path.name}")

    # 2-1. 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 2-2. 기존 billsCommitteeStatistics가 있으면 삭제
    if 'billsCommitteeStatistics' in data:
        print(f"  - 기존 billsCommitteeStatistics 컬럼을 삭제합니다.")
        del data['billsCommitteeStatistics']

    # 2-3. bills 데이터 수집
    bills = data.get('bills', [])
    bills_count = len(bills)

    # 👉 bills가 아예 없는 경우 경고 출력 + 기록
    if bills_count == 0:
        print(f"  ⚠️  경고: 이 파일은 bills 데이터가 없습니다!")
        empty_bills_files.append(file_path.name)

    counter = Counter()
    for bill in bills:
        nth = (bill.get('nth') or '').replace('제', '').replace('대', '')
        committee = (bill.get('committee') or '').strip()
        if nth and committee:
            counter[(nth, committee)] += 1

    # 2-4. Counter를 리스트로 변환
    bills_statistics = [
        {
            'nth': f"{nth}대",
            'name': committee,
            'value': count
        }
        for (nth, committee), count in counter.items()
    ]

    statistics_count = len(bills_statistics)

    # 2-5. 결과를 데이터에 추가
    data['billsCommitteeStatistics'] = bills_statistics

    # 2-6. 결과 파일 저장 (원래 파일 덮어쓰기)
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"  - 저장 완료: {file_path.name}\n")

    # 2-7. 요약 기록 저장
    summary_records.append({
        '파일명': file_path.name,
        'bills 항목 수': bills_count,
        '통계 항목 수': statistics_count
    })

# 3. 요약 테이블 출력
summary_df = pd.DataFrame(summary_records)

# 👉 '통계 항목 수' 기준 내림차순 정렬
summary_df = summary_df.sort_values(by='통계 항목 수', ascending=False)

print("✅ 처리 결과 요약:")
display(summary_df)

# 4. bills가 없는 파일 목록 출력
if empty_bills_files:
    print("\n⚠️ bills 데이터가 없는 파일 목록:")
    for filename in empty_bills_files:
        print(f" - {filename}")
else:
    print("\n✅ 모든 파일에 bills 데이터가 존재합니다!")


총 29개의 JSON 파일을 처리합니다.

처리 중: KangSeungKyoo.json
  - 기존 billsCommitteeStatistics 컬럼을 삭제합니다.
  - 저장 완료: KangSeungKyoo.json

처리 중: KANGYUJUNG.json
  - 기존 billsCommitteeStatistics 컬럼을 삭제합니다.
  - 저장 완료: KANGYUJUNG.json

처리 중: KANGKYUNGSOOK.json
  - 기존 billsCommitteeStatistics 컬럼을 삭제합니다.
  - 저장 완료: KANGKYUNGSOOK.json

처리 중: KWAKKYUTAEK.json
  - 기존 billsCommitteeStatistics 컬럼을 삭제합니다.
  - 저장 완료: KWAKKYUTAEK.json

처리 중: KIMNAMGEUN.json
  - 기존 billsCommitteeStatistics 컬럼을 삭제합니다.
  - 저장 완료: KIMNAMGEUN.json

처리 중: KANGSUNYOUNG.json
  - 기존 billsCommitteeStatistics 컬럼을 삭제합니다.
  - 저장 완료: KANGSUNYOUNG.json

처리 중: KWONHYANGYUP.json
  - 기존 billsCommitteeStatistics 컬럼을 삭제합니다.
  - 저장 완료: KWONHYANGYUP.json

처리 중: KANGMINKUK.json
  - 기존 billsCommitteeStatistics 컬럼을 삭제합니다.
  - 저장 완료: KANGMINKUK.json

처리 중: KWONYOUNGSE.json
  - 기존 billsCommitteeStatistics 컬럼을 삭제합니다.
  - 저장 완료: KWONYOUNGSE.json

처리 중: KIMDAISIK.json
  - 기존 billsCommitteeStatistics 컬럼을 삭제합니다.
  - 저장 완료: KIMDAISIK.json

처리 중: KANGSUNWOO.json
  

,파일명,bills 항목 수,통계 항목 수
22,KIMGIHYEON.json,120,38
20,KWEONSEONGDONG.json,190,36
15,KWONCHILSEUNG.json,250,36
14,KANGHOONSIK.json,160,27
13,KANGDAESIK.json,130,21
27,KANGDEUKGU.json,140,19
18,KUJAKEUN.json,167,19
8,KWONYOUNGSE.json,75,18
24,KOMINJUNG.json,60,16
12,KANGJUNHYEON.json,86,15



✅ 모든 파일에 bills 데이터가 존재합니다!
